# Python Langchain, FAISS

- LangChain 사용 위한 환경 설정과 FAISS로 Vector DB를 구성하는 실습

## 설치 및 기본 설정

'pip install langchain langchain-openai faiss-cpu'  
  
- LangChain, OpenAI, FAISS 패키지 설치.  
  
- 설치 후, OpenAI API 키 설정.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='key.env')
api_key=os.getenv("GROUP_API_KEY")

## LangChain 기본 개념

1. __언어 모델 초기화__

- GPT-4 모델을 LangChain 통해 사용. ChatOpenAI를 이용해 초기화, invoke 메서드로 메시지 전달해 응답 받아옴.

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4", api_key=api_key)

# 모델에 메시지 전달
response = model.invoke([HumanMessage(content="안녕하세요, 무엇을 도와드릴까요?")])
print(response.content)

사용자의 요청을 듣고 있습니다. 무엇을 도와드릴까요?


2. __프롬프트 템플릿 사용하기__

- 다양한 입력을 받아 메시지 생성에 도움을 줌. 영어 문장을 다른 언어로 번역하는 프롬프트 정의.

In [3]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 메시지 설정
system_template = "Translate the following sentence from English to {language}:"

# 사용자 텍스트 입력
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{text}")
])

# 프롬프트 생성
result = prompt_template.invoke({"language": "French", "text": "How are you?"})
print(result.to_messages())


[SystemMessage(content='Translate the following sentence from English to French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})]


## LangChain Expression Language(LCEL)로 체인 연결

- 여러 컴포넌트를 체인으로 연결해서 데이터 흐름을 통제하는 LCEL 사용.

In [4]:
from langchain_core.output_parsers import StrOutputParser

# 응답을 파싱하는 파서 초기화
parser = StrOutputParser()

# 템플릿, 모델, 파서를 체인으로 연결
chain = prompt_template | model | parser

# 체인 실행
response = chain.invoke({"language": "Spanish", "text": "Where is the library?"})
print(response)

¿Dónde está la biblioteca?


## FAISS 활용 Vector DB 구성 및 쿼리

- FAISS는 벡터 유사성 검색 위한 라이브러리. OpenAIEmbeddings로 텍스트를 벡터로 변환해서 FAISS index에 저장.

1. OpenAI Embedding model로 Vector Embedding 생성

In [5]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

2. FAISS index 초기화

In [6]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# FAISS 인덱스 생성
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

3. Vector DB에 문서 추가

In [7]:
from langchain_core.documents import Document
from uuid import uuid4

# 문서 생성
documents = [
    Document(page_content="LangChain을 사용해 프로젝트를 구축하고 있습니다!", metadata={"source": "tweet"}),
    Document(page_content="내일 날씨는 맑고 따뜻할 예정입니다.", metadata={"source": "news"}),
    Document(page_content="오늘 아침에는 팬케이크와 계란을 먹었어요.", metadata={"source": "personal"}),
    Document(page_content="주식 시장이 경기 침체 우려로 하락 중입니다.", metadata={"source": "news"}),
]

# 고유 ID 생성 및 문서 추가
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['a8399d99-da34-4144-b9d5-178db922c535',
 '3464d19d-58ab-4894-8e6c-60a99920e8eb',
 '691cbde2-f432-4586-b095-3c118c028c55',
 'ae042198-fbfc-4c1b-a6a0-262cf78f1ebd']

4. Vector DB Query

- 유사성 검색을 통해 특정 쿼리와 유사한 문서를 검색.

In [8]:
# 기본 유사성 검색
results = vector_store.similarity_search("내일 날씨는 어떨까요?", k=2, filter={"source": "news"})
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

# 점수와 함께 유사성 검색
results_with_scores = vector_store.similarity_search_with_score("LangChain에 대해 이야기해주세요.", k=2, filter={"source": "tweet"})
for res, score in results_with_scores:
    print(f"* [SIM={score:.3f}] {res.page_content} [{res.metadata}]")

* 내일 날씨는 맑고 따뜻할 예정입니다. [{'source': 'news'}]
* 주식 시장이 경기 침체 우려로 하락 중입니다. [{'source': 'news'}]
* [SIM=0.159] LangChain을 사용해 프로젝트를 구축하고 있습니다! [{'source': 'tweet'}]


## RAG Chain에 FAISS 통합

- RAG 체인을 구성하여 검색된 문서를 바탕으로 질문에 응답할 수 있도록 구성.

1. FAISS를 Retriever로 변환해 RAG 체인에서 사용.

In [9]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

2. RAG Chain 생성

- LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성함.

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])


class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model

# 질문 실행 및 각 단계 출력 확인
response = rag_chain_debug.invoke("내일 날씨는?")
print("Final Response:")
print(response.content)

Debug Output: 내일 날씨는?
Debug Output: {'context': [Document(metadata={'source': 'news'}, page_content='내일 날씨는 맑고 따뜻할 예정입니다.')], 'question': '내일 날씨는?'}
Final Response:
맑고 따뜻할 예정입니다.


## FAISS index의 저장 및 로드

In [12]:
# 인덱스 저장
vector_store.save_local("faiss_index")

# 저장된 인덱스 로드
new_vector_store = FAISS.load_local("faiss_index", embeddings)

ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

## FAISS DB 병합

In [13]:
db1 = FAISS.from_texts(["문서 1 내용"], embeddings)
db2 = FAISS.from_texts(["문서 2 내용"], embeddings)

# 병합
db1.merge_from(db2)